In [1]:
import pandas as pd
import tensorflow
import numpy as np
import tensorflow.keras as keras
import os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Mounted at /content/drive


Saving kaggle.json to kaggle.json


In [2]:
model_bin = keras.models.load_model('/content/drive/MyDrive/Pro2/models/model_seen_notseen_cNN_2.hdf5')
model_class = keras.models.load_model('/content/drive/MyDrive/Pro2/models/model_11_classes_LSTM.hdf5')

label_encodings=pd.read_csv('/content/drive/MyDrive/Pro2/models/label_encodings.csv')

X_test=np.load('/content/drive/MyDrive/Pro2/test_set/X_test.npy')
fnames=pd.read_csv('/content/drive/MyDrive/Pro2/test_set/fnames.csv')

In [3]:
#Predictions for binary class model need to predict also via 11 class model
%%time
y_pred=model_bin.predict(X_test).argmax(axis=1)

CPU times: user 2min 36s, sys: 2.6 s, total: 2min 39s
Wall time: 3min 23s


In [4]:
#Predictions for class model
%%time
y_pred_class=model_class.predict(X_test).argmax(axis=1)

CPU times: user 2min 40s, sys: 1.62 s, total: 2min 42s
Wall time: 2min 40s


In [5]:
predictions = np.where(y_pred == 1, y_pred_class, 9)

In [ ]:
label_encodings

,index,name
0,0,down
1,1,go
2,2,left
3,3,no
4,4,off
5,5,on
6,6,right
7,7,silence
8,8,stop
9,9,unknown


In [6]:
labels_enc_dict=dict(zip(label_encodings['index'],label_encodings['name']))
y_submit=pd.DataFrame(data=predictions, columns=["label"]).replace(labels_enc_dict).reset_index()
y_submit.columns=['fname','label'] 
y_submit["fname"]=fnames['0']
y_submit.to_csv('/content/drive/MyDrive/Pro2/y_submit.csv',index=False)
y_submit.to_csv('/content/my_submission_CNN.csv',index=False)

In [ ]:
y_submit['label'].value_counts()

unknown    158404
go             66
no             42
down           24
stop            2
Name: label, dtype: int64

In [7]:
#run this cell if you want to submit
!kaggle competitions submit tensorflow-speech-recognition-challenge -f my_submission_CNN.csv -m "Double CNN"

100% 3.89M/3.89M [00:02<00:00, 1.63MB/s]
Successfully submitted to TensorFlow Speech Recognition Challenge